# Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr

from pystac_client import Client  # To access STAC catalogs

import planetary_computer  # To sign items from the MPC STAC catalog 

from IPython.display import Image  # To nicely display images

# Data access

The datasets hosted by the Planetary Computer are available from Azure Blob Storage. We'll use pystac-client to search the Planetary Computer's STAC API for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a modifier so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See Reading from the STAC API and Using tokens for data access for more.

In [2]:
# Access MPC catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

catalog.get_collections()

<generator object Client.get_collections at 0x7f17638a3ab0>

# Select a region and find STAC Items

Let's look at an area centered on the Maricopa County, part of the Phoenix metropolitan area

In [4]:
bbox_of_interest = [-112.826843, 32.974108, -111.184387, 33.863574]

# Temporal range of interest
time_range = "2017-01-01/2020-01-01"

# NCEAS bounding box (as a GeoJSON)
bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            [-112.826843, 32.974108],  # Bottom-left corner
            [-112.826843, 33.863574],  # Top-left corner
            [-111.184387, 33.863574],  # Top-right corner
            [-111.184387, 32.974108],  # Bottom-right corner
            [-112.826843, 32.974108]   # Back to bottom-left corner to close the polygon
        ]
    ],
}

# Catalog search
search = catalog.search(
    collections = ["io-biodiversity"],
    intersects = bbox,
    datetime = time_range)
search

In [5]:
# Retrieve search items
items = search.item_collection()
len(items)

items

In [9]:
# Get first item in the catalog search
item = items[1]
type(item)


pystac.item.Item

In [10]:
# Print item ID and properties
print('ID:' , item.id)
item.properties

ID: bii_2019_34.74464974521749_-115.38597824385106_cog


{'datetime': None,
 'proj:epsg': 4326,
 'proj:shape': [7992, 7992],
 'end_datetime': '2019-12-31T23:59:59Z',
 'proj:transform': [0.0008983152841195215,
  0.0,
  -115.38597824385106,
  0.0,
  -0.0008983152841195215,
  34.74464974521749,
  0.0,
  0.0,
  1.0],
 'start_datetime': '2019-01-01T00:00:00Z'}

In [12]:
for key in item.assets.keys():
    print(key, '--', item.assets[key].title)

data -- Biodiversity Intactness
tilejson -- TileJSON with default rendering
rendered_preview -- Rendered preview


In [17]:
# Plot rendered preview
Image(url=item.assets['rendered_preview'].href, width=500)

In [14]:
# Retrieve search items
items = search.item_collection()
len(items)

4

In [25]:
# Get first item in the catalog search
item = items[0]
item
#item.assets['image']

<Item id=bii_2020_34.74464974521749_-115.38597824385106_cog>

In [27]:
biodiversity = rioxr.open_rasterio(item.assets['data'].href)
biodiversity


<xarray.DataArray (band: 1, y: 7992, x: 7992)> Size: 255MB
[63872064 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 64kB -115.4 -115.4 -115.4 ... -108.2 -108.2 -108.2
  * y            (y) float64 64kB 34.74 34.74 34.74 34.74 ... 27.57 27.57 27.57
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [30]:
# Remove length 1 dimension (band)
biodiversity = biodiversity.squeeze().drop_vars('band')
print("Sizes of dimensions:", dict(lulc.sizes))

Sizes of dimensions: {'y': 7992, 'x': 7992}


In [ ]:
# Create GeoDataFrame from raster bounding box
biodiversity_bbox = gpd.GeoDataFrame(geometry = [box(*lulc.rio.bounds())],
                             crs = lulc.rio.crs)

ca = gpd.read_file(os.path.join('data',
                                'ca_state_boundary',   
                                'ca_state_boundary.shp'))

# ------------------------------------------------------------------
# Plot raster boundary, fire perimeter, and CA boundary
fig, ax = plt.subplots()
ca.plot(ax=ax, color='white', edgecolor ='black')

# Reproject lulc_bbox and fire perimeter to match CA crs
lulc_bbox.to_crs(ca.crs).plot(ax=ax, alpha=0.3)  
thomas_fire.to_crs(ca.crs).plot(ax=ax, color='red')

plt.show()

# Available Assets & Item Properties

Our search returned four STAC Items. We can tell from their IDs that that they contain data for the same area but for different times, specifically the years 2017 through 2020. Let's display the available assets and properties for the 2017 Item.

In [11]:
asset_table = rich.table.Table("Asset Key", "Asset Title")
for key, value in items[-1].assets.items():
    asset_table.add_row(key, value.title)
asset_table

NameError: name 'rich' is not defined

In [ ]:
property_table = rich.table.Table("Property Name", "Property Value")
for key, value in sorted(items[-1].properties.items()):
    property_table.add_row(key, str(value))
property_table

# Load the Data and Plot

We are interested in the "data" asset. We'll use the stackstac library to read the data assets for the four Items into a single `xarray.DataArray` .

In [ ]:
stack = (
    stackstac.stack(items, bounds_latlon=bbox_of_interest, assets=["data"])
    .assign_coords(
        time=pd.to_datetime([item.properties["start_datetime"] for item in items])
        .tz_convert(None)
        .to_numpy()
    )
    .sortby("time")
)
stack.name = "Biodiversity Intactness"
stack

At this point we haven't loaded any data into memory yet. Let's drop the single band dimension and load the data by calling `.compute()` .

In [ ]:
data_array = stack.squeeze().compute()


Now we can plot the data time series in a grid.

In [ ]:
grid = data_array.plot(col="time", cmap="Greens", robust=True)
grid;

It's not easy to see the changes in a static plot, so let's make a gif that loops through the years.

In [ ]:
gif(data_array, fps=1, cmap="Greens", robust=True)